In [ ]:
## cross validation 


# Define feature sets
X_clinical = df[clinical_vars]                  # Clinical variables only
X_overlap = df[[overlap_var]]                    # Overlap variable only
X_clinical_overlap = df[clinical_vars + [overlap_var]]  # Clinical + Overlap combined
y = df[target_var]

# Define cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Store AUC scores and ROC curve points
clinical_auc = []
overlap_auc = []
clinical_overlap_auc = []

tpr_clinical_list = []
tpr_overlap_list = []
tpr_clinical_overlap_list = []

fpr_clinical_list = []
fpr_overlap_list = []
fpr_clinical_overlap_list = []


mean_fpr = np.linspace(0, 1, 100)

# Perform cross-validation
y_pred=np.zeros((len(y),3))

model='xg'

for train_idx, test_idx in cv.split(X_clinical, y):
    X_train_clinical, X_test_clinical = X_clinical.iloc[train_idx], X_clinical.iloc[test_idx]
    X_train_overlap, X_test_overlap = X_overlap.iloc[train_idx], X_overlap.iloc[test_idx]
    X_train_clinical_overlap, X_test_clinical_overlap = X_clinical_overlap.iloc[train_idx], X_clinical_overlap.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    # Train logistic regression models
    if 'forest' in model:
        model_clinical = RandomForestClassifier(random_state=42)
        model_overlap = RandomForestClassifier(random_state=42)
        model_clinical_overlap = RandomForestClassifier(random_state=42)
    elif 'xg' in model:
        model_clinical = XGBClassifier(base_score=0.5, booster='gbtree', gamma=0, 
                                    max_depth=1, eval_metric='error', subsample=0.9,
                                    objective='binary:logistic', use_label_encoder=False)
        model_overlap = XGBClassifier(base_score=0.5, booster='gbtree', gamma=0, 
                                    max_depth=1, eval_metric='error', subsample=0.9,
                                    objective='binary:logistic', use_label_encoder=False)
        model_clinical_overlap = XGBClassifier(base_score=0.5, booster='gbtree', gamma=0, 
                                            max_depth=1, eval_metric='error', subsample=0.9,
                                            objective='binary:logistic', use_label_encoder=False)
    elif 'logis' in model:
        
        model_clinical = LogisticRegression(max_iter=1000, random_state=42)
        model_overlap = LogisticRegression(max_iter=1000, random_state=42)
        model_clinical_overlap = LogisticRegression(max_iter=1000, random_state=42)
    else:
        print("Model name not recognised")
        break

    model_clinical.fit(X_train_clinical, y_train)
    model_overlap.fit(X_train_overlap, y_train)
    model_clinical_overlap.fit(X_train_clinical_overlap, y_train)
    
    # Get probabilities for ROC calculation
    y_pred_clinical = model_clinical.predict_proba(X_test_clinical)[:, 1]
    y_pred_overlap = model_overlap.predict_proba(X_test_overlap)[:, 1]
    y_pred_clinical_overlap = model_clinical_overlap.predict_proba(X_test_clinical_overlap)[:, 1]
    y_pred[test_idx,0] = y_pred_clinical
    y_pred[test_idx,1] = y_pred_overlap
    y_pred[test_idx,2] = y_pred_clinical_overlap
 


plt.figure(figsize=(8, 6))
fpr_clinical, tpr_clinical, _ = roc_curve(y, y_pred[:,0])
fpr_overlap, tpr_overlap, _ = roc_curve(y, y_pred[:,1])
fpr_clinical_overlap, tpr_clinical_overlap, _ = roc_curve(y, y_pred[:,2])
clinical_auc = roc_auc_score(y, y_pred[:,0])
overlap_auc = roc_auc_score(y, y_pred[:,1])
clinical_overlap_auc = roc_auc_score(y, y_pred[:,2])

plt.plot(fpr_clinical, tpr_clinical, label=f'Clinical only (AUC = {clinical_auc:.3f})', linestyle='--')
plt.plot(fpr_overlap, tpr_overlap, label=f'Overlap only (AUC = {overlap_auc:.3f})', linestyle='-.')
plt.plot(fpr_clinical_overlap, tpr_clinical_overlap, label=f'Clinical + Overlap (AUC = {clinical_overlap_auc:.3f})', linestyle='-')

# Random classifier line
plt.plot([0, 1], [0, 1], linestyle="--", color="gray", label="Random classifier (AUC = 0.5)")

plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.title("Cross-Validation Logistic Regression Model ")
plt.legend(loc='lower right')
plt.grid()
plt.show()
plt.figure(figsize=(8, 6))


In [ ]:
## AUC distribution 


# Define model pairs for significance testing
model_pairs = [
    ("Logistic_Clinical", "Logistic_Overlap"),
    ("Logistic_Clinical", "Logistic_Combined"),
    ("Logistic_Overlap", "Logistic_Combined"),
    ("XGBoost_Clinical", "XGBoost_Overlap"),
    ("XGBoost_Clinical", "XGBoost_Combined"),
    ("XGBoost_Overlap", "XGBoost_Combined"),
    ("XGBoost_Combined", "Logistic_Combined")
]

# Store t-test results
p_values = {}
for model1, model2 in model_pairs:
    _, p_value = ttest_rel(auc_scores[model1], auc_scores[model2])
    p_values[(model1, model2)] = p_value

# Define x positions
x_labels = list(auc_scores.keys())
x_pos = np.arange(len(x_labels))

# Plot scatter and boxplots
plt.figure(figsize=(10, 6))

for i, key in enumerate(x_labels):
    plt.scatter([x_pos[i]] * 10, auc_scores[key], label=key, alpha=0.7)
    


# Function to get significance stars
def get_significance_stars(p):
    if p < 0.001:
        return "***"
    elif p < 0.01:
        return "**"
    elif p < 0.05:
        return "*"
    else:
        return ""

# Annotate significance
y_offset = 0.01  # Adjust for better visibility
for (model1, model2), p_val in p_values.items():
    if p_val < 0.05:  # Only show significant comparisons
        x1, x2 = x_labels.index(model1), x_labels.index(model2)
        y_max = max(max(auc_scores[model1]), max(auc_scores[model2])) + y_offset
        
        # Adjust the y_max position for the specific pair: "XGBoost_Clinical" vs. "XGBoost_Combined"
        if model1 == "XGBoost_Clinical" and model2 == "XGBoost_Combined":
            y_max += 0.03  # Shift upward by 0.05 to make space

        plt.plot([x1, x1, x2, x2], [y_max, y_max + 0.01, y_max + 0.01, y_max], color='black', lw=1.5)
        plt.text((x1 + x2) / 2, y_max + 0.015, get_significance_stars(p_val), ha='center', fontsize=12)
x_labels = ["clinical","resection extent","combined","clinical","resection extent","combined"]
plt.text(1,0.43,'logistic regression',ha='center')
plt.text(4,0.43,'XGboost',ha='center')
# Final plot settings
plt.xticks(x_pos, x_labels, rotation=30, ha="right")
plt.ylim(0.5, 0.85)
plt.ylabel("AUC Score")
plt.title("")
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.show()